In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import feature_manager as fma
import rl.env_simple_crypto_trade as env
import importlib
import rl.models as rla
import config as cf

In [2]:
importlib.reload(fma)

fm = fma.FeatureManager(target_col="trade_signal")
fm.import_trading_data(symbol="BTCUSDT",trade_timeframe="4h")
fm.import_macro_data(symbol="BTCUSDT",macro_timeframe="1d")
fm.import_super_data(symbol="BTCUSDT",super_timeframe="1w")

fm.build_features(
    lags = 1,
    macro_lags = 1,
    super_lags = 1,
    features = cf.BITCOIN_EXTERNAL_INDICATORS + cf.TRADING_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
    macro_features = cf.MACRO_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
    super_features = cf.SUPER_TA_INDICATORS + cf.CANDLESTICK_INDICATORS,
)

Imported trading data from ../data/BTCUSDT-4h.csv with 11760 rows
Imported macro data from ../data/BTCUSDT-1d.csv with 1963 rows
Imported super macro data from ../data/BTCUSDT-1w.csv with 247 rows
Calculating external features ...
Calculating TA indicators of trade timeframe
Calculating TA indicators of macro timeframe
Calculating TA indicators of super timeframe
Calculating candlestick of trade timeframe
Calculating candlestick of macro timeframe
Calculating candlestick of super timeframe

Adding features for trade timeframe with lags 1: hashrate, fed_rate, gold, nasdaq, sp500, google_trend, sma_3_10, sma_7_30, sma_14_50, cci7, cci14, cci30, dx7, dx14, dx30, up_bb, low_bb, CDL2CROWS, CDL3BLACKCROWS, CDL3INSIDE, CDL3LINESTRIKE, CDL3OUTSIDE, CDL3STARSINSOUTH, CDL3WHITESOLDIERS, CDLABANDONEDBABY, CDLADVANCEBLOCK, CDLBELTHOLD, CDLBREAKAWAY, CDLCLOSINGMARUBOZU, CDLCONCEALBABYSWALL, CDLCOUNTERATTACK, CDLDARKCLOUDCOVER, CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI, CDLENGULFING, CDLEVENINGDOJISTAR

In [3]:
train = fm.df.iloc[-4000:-2000]
trade = fm.df.iloc[-2000:]

In [4]:
importlib.reload(env)

env_kwargs = cf.TRADE_ENV_PARAMETER
state_space = 3 + len(fm.cols)

train_env = env.CryptoTradingEnv(
    df = train, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

trade_env = env.CryptoTradingEnv(
    df = trade, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)
full_env = env.CryptoTradingEnv(
    df = fm.df, 
    state_space=state_space,
    indicators=fm.cols,
    **env_kwargs)

In [5]:
importlib.reload(rla)
agent = rla.DRLTradeAgent(env=full_env)

In [6]:
model_ppo = agent.get_model(model_name="ppo",gamma=0.995,seed=100)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
selected_model = model_ppo
selected_model = agent.train_model(selected_model,
    total_timesteps=300_000,
    checkpoint=True,
    checkpoint_subdir_name="gamma_995",
    save_frequency=5_000,
)